In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-data/jinteng_train_data_no_test.txt
/kaggle/input/train-data/jinteng_train_data_have_test_mrc.txt
/kaggle/input/competition/competition_tag_extract_pretrain_data.txt
/kaggle/input/pretrain-model/pretrain/pretrain/config.json
/kaggle/input/pretrain-model/pretrain/pretrain/pytorch_model.bin
/kaggle/input/pretrain-model/pretrain/pretrain/vocab.txt
/kaggle/input/pretrain-model/pretrain_model_0811/pretrain_model_0811/config.json
/kaggle/input/pretrain-model/pretrain_model_0811/pretrain_model_0811/readme.txt
/kaggle/input/pretrain-model/pretrain_model_0811/pretrain_model_0811/pytorch_model.bin
/kaggle/input/pretrain-model/pretrain_model_0811/pretrain_model_0811/vocab.txt


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("kate")

import wandb
import os
os.system("wandb login 525037aaae74525566dca1246bce266cc1c1330b")
import wandb
wandb.init(project="kate_project", entity="catherine1015")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: catherine1015. Use `wandb login --relogin` to force relogin


In [3]:
import os
import csv
from transformers import  BertTokenizer, WEIGHTS_NAME,TrainingArguments,BertConfig,BertForMaskedLM
import torch
from datasets import load_dataset,Dataset
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    LineByLineTextDataset
)



In [4]:
bert_file = 'hfl/chinese-roberta-wwm-ext'
#bert_file = '../input/pretrain-model/pretrain_model_0811/pretrain_model_0811'
#data_path = '/kaggle/input/train-data/jinteng_train_data_no_test.txt'
#data_path = '../input/trian-data-512/unsup_train_data_large_part_512.txt/unsup_train_data_large_part_512.txt'
#data_path = '../input/train-data/jinteng_train_data_have_test_mrc.txt'
data_path = '../input/competition/competition_tag_extract_pretrain_data.txt'
#data_dir = '/data/keraszhou/jinteng_pretrain_bert_model/jinteng_pretrain_bert_model/data/'
#outputs= data_dir + 'outputs/model/'
outputs = './'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('----device', device)

----device cuda


In [5]:

config = BertConfig.from_pretrained(bert_file)
tokenizer = BertTokenizer.from_pretrained(bert_file)
model = BertForMaskedLM.from_pretrained(bert_file).to(device)
print('-------train load over')


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-------train load over


In [6]:
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [7]:
train_dataset = LineByLineTextDataset(tokenizer=tokenizer,file_path=data_path, block_size=128)
# MLM模型的数据DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
print('-------data mask')
# 训练参数
pretrain_batch_size = 64
num_train_epochs = 10  # 这个数据量要小
training_args = TrainingArguments(
    output_dir=outputs, overwrite_output_dir=True, num_train_epochs=num_train_epochs, learning_rate=6e-5,
    per_device_train_batch_size=pretrain_batch_size, save_total_limit=10)# save_steps=10000

# 通过Trainer接口训练模型
trainer = Trainer(
    model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)


/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


-------data mask


In [8]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}
#wandb.log({"loss":trainer._total_loss_scalar,"epoch": trainer.state.global_step})
# Optional
wandb.watch(model)
print('------wandb config')

------wandb config


In [9]:
# 开始训练
trainer.train() # 如果是字符串

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12662
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1980
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
500,1.599100
1000,1.446100
1500,1.376300


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./checkpoint-1500
Configuration saved in ./checkpoint-1500/config.json
Model weights saved in ./checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1980, training_loss=1.442647976345486, metrics={'train_runtime': 1646.6354, 'train_samples_per_second': 76.896, 'train_steps_per_second': 1.202, 'total_flos': 8330834837544960.0, 'train_loss': 1.442647976345486, 'epoch': 10.0})

In [10]:
trainer.save_model(outputs)

Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
